In [2]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import random

def create_data_model():
    num_cities = 10
    random.seed(42)  

    distance_matrix = [[0]*num_cities for _ in range(num_cities)]
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            dist = random.randint(10, 100)
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist

    return {
        'distance_matrix': distance_matrix,
        'num_vehicles': 1,
        'depot': 0  # Start at city 0
    }

def solve_tsp():
    data = create_data_model()

    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    routing = pywrapcp.RoutingModel(manager)

    # Cost callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # TSP parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print("Optimal Route:")
        index = routing.Start(0)
        route = []
        route_distance = 0
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(prev_index, index, 0)
        route.append(manager.IndexToNode(index))  # return to depot
        print(" -->  ".join(map(str, route)))
        print("Total Distance:", route_distance)
    else:
        print("No solution found.")

solve_tsp()


Optimal Route:
0 -->  3 -->  1 -->  7 -->  9 -->  6 -->  4 -->  5 -->  8 -->  2 -->  0
Total Distance: 233
